# Verifying the MLOps environment on GCP

This notebook verifies the MLOps environment provisioned on GCP
1. Test using the local MLflow server in AI Notebooks instance in log entries to the Cloud SQL
2. Test deploying and running an Airflow workflow on Composer that uses MLflow server on GKE to log entries to the Cloud SQL

## 1. Running a local MLflow experiment
We implement a simple Scikit-learn model training routine, and examine the logged entries in Cloud SQL and produced articats in Cloud Storage through MLflow tracking.

In [ ]:
import os
import re
import mlflow
import mlflow.sklearn
import numpy as np
from sklearn.linear_model import LogisticRegression
import pymysql
from IPython.core.display import display, HTML

In [ ]:
mlflow_tracking_uri = mlflow.get_tracking_uri()
MLFLOW_EXPERIMENTS_URI = os.environ['MLFLOW_EXPERIMENTS_URI']

print("MLflow tracking server URI: {}".format(mlflow_tracking_uri))
print("MLflow artifacts store root: {}".format(MLFLOW_EXPERIMENTS_URI))
print("MLflow SQL connction name: {}".format(os.environ['MLFLOW_SQL_CONNECTION_NAME']))
print("MLflow SQL connction string: {}".format(os.environ['MLFLOW_SQL_CONNECTION_STR']))
print("Cloud Composer name: {}".format(os.environ['MLOPS_COMPOSER_NAME']))
print("Cloud Composer instance region: {}".format(os.environ['MLOPS_REGION']))

display(HTML('<hr>You can check results of this test in MLflow and GCS folder:'))
display(HTML('<h4><a href="{}" rel="noopener noreferrer" target="_blank">Click to open MLflow UI</a></h4>'.format(os.environ['MLFLOW_TRACKING_EXTERNAL_URI'])))
display(HTML('<h4><a href="https://console.cloud.google.com/storage/browser/{}" rel="noopener noreferrer" target="_blank">Click to open GCS folder</a></h4>'.format(MLFLOW_EXPERIMENTS_URI.replace('gs://',''))))


### 1.1. Training a simple Scikit-learn model from Notebook environment

In [ ]:
experiment_name = "notebooks-test"
mlflow.set_experiment(experiment_name)

with mlflow.start_run(nested=True):
    X = np.array([-2, -1, 0, 1, 2, 1]).reshape(-1, 1)
    y = np.array([0, 0, 1, 1, 1, 0])
    lr = LogisticRegression()
    lr.fit(X, y)
    score = lr.score(X, y)
    print("Score: %s" % score)
    mlflow.log_metric("score", score)
    mlflow.sklearn.log_model(lr, "model")
    print("Model saved in run %s" % mlflow.active_run().info.run_uuid)
    current_model=mlflow.get_artifact_uri('model')

### 1.2. Query the Mlfow entries from Cloud SQL

In [ ]:
sqlauth=re.search('mysql\\+pymysql://(?P<user>.*):(?P<psw>.*)@127.0.0.1:3306/mlflow', os.environ['MLFLOW_SQL_CONNECTION_STR'],re.DOTALL)
connection = pymysql.connect(
    host='127.0.0.1',
    port=3306,
    database='mlflow',
    user=sqlauth.group('user'),
    passwd=sqlauth.group('psw')
)

#### List tables
You should see a list of table names like 'experiments','metrics','model_versions','runs'

In [ ]:
cursor = connection.cursor()   
cursor.execute("SHOW TABLES")
for entry in cursor:
    print(entry[0])

#### Retrieve experiment

In [ ]:
cursor.execute("SELECT * FROM experiments where name='{}' ORDER BY experiment_id desc LIMIT 1".format(experiment_name))
if cursor.rowcount == 0:
    print("Experiment not found")
else:
    experiment_id = list(cursor)[0][0]
    print("'{}' experiment ID: {}".format(experiment_name, experiment_id))

#### Query runs

In [ ]:
cursor.execute("SELECT * FROM runs where experiment_id={} ORDER BY start_time desc LIMIT 1".format(experiment_id))
if cursor.rowcount == 0:
    print("No runs found")
else:
    entity=list(cursor)[0]
    run_uuid = entity[0]
    print("Last run id of '{}' experiment is: {}\n".format(experiment_name, run_uuid))
    print(entity)

#### Query metrics

In [ ]:
cursor.execute("SELECT * FROM metrics where run_uuid = '{}'".format(run_uuid))
for entry in cursor:
    print(entry)

### 1.3. List the artifacts in Cloud Storage

In [ ]:
!gsutil ls {current_model}

## 2. Submitting a workflow to Composer

We implement a one-step Airflow workflow that trains a Scikit-learn model, and examine the logged entries in Cloud SQL and produced articats in Cloud Storage through MLflow tracking.

In [ ]:
COMPOSER_NAME=os.environ['MLOPS_COMPOSER_NAME']
REGION=os.environ['MLOPS_REGION']

### 2.1. Writing the Airflow workflow

In [ ]:
%%writefile test-sklearn-mlflow.py

import airflow
import mlflow
import mlflow.sklearn
import numpy as np
from datetime import timedelta
from sklearn.linear_model import LogisticRegression
from airflow.operators import PythonOperator

def train_model(**kwargs):

    print("Train lr model step started...")
    print("MLflow tracking uri: {}".format(mlflow.get_tracking_uri()))
    mlflow.set_experiment("airflow-test")
    with mlflow.start_run(nested=True):
        X = np.array([-2, -1, 0, 1, 2, 1]).reshape(-1, 1)
        y = np.array([0, 0, 1, 1, 1, 0])
        lr = LogisticRegression()
        lr.fit(X, y)
        score = lr.score(X, y)
        print("Score: %s" % score)
        mlflow.log_metric("score", score)
        mlflow.sklearn.log_model(lr, "model")
        print("Model saved in run %s" % mlflow.active_run().info.run_uuid)
    print("Train lr model step finished.")
    
default_args = {
    'retries': 1,
    'start_date': airflow.utils.dates.days_ago(0)
}

with airflow.DAG(
    'test_sklearn_mlflow',
    default_args=default_args,
    schedule_interval=None,
    dagrun_timeout=timedelta(minutes=20)) as dag:
    
    train_model_op = PythonOperator(
        task_id='train_sklearn_model',
        provide_context=True,
        python_callable=train_model
    )

### 2.2. Uploading the Airflow workflow

In [ ]:
!gcloud composer environments storage dags import \
  --environment {COMPOSER_NAME}  --location {REGION} \
  --source test-sklearn-mlflow.py

In [ ]:
!gcloud composer environments storage dags list \
  --environment {COMPOSER_NAME}  --location {REGION}

### 2.3. Triggering the workflow

Please wait for 30-60 seconds before triggering the workflow at the first Airflow Dag import

In [ ]:
!gcloud composer environments run {COMPOSER_NAME} \
    --location {REGION} unpause -- test_sklearn_mlflow

In [ ]:
!gcloud composer environments run {COMPOSER_NAME} \
    --location {REGION} trigger_dag -- test_sklearn_mlflow

### 2.4. Query the MLfow entries from Cloud SQL

In [ ]:
cursor = connection.cursor()

#### Retrieve experiment

In [ ]:
experiment_name = "airflow-test"
cursor.execute("SELECT * FROM experiments where name='{}' ORDER BY experiment_id desc LIMIT 1".format(experiment_name))
if cursor.rowcount == 0:
    print("Experiment not found")
else:
    experiment_id = list(cursor)[0][0]
    print("'{}' experiment ID: {}".format(experiment_name, experiment_id))

#### Query runs

In [ ]:
cursor.execute("SELECT * FROM runs where experiment_id={} ORDER BY start_time desc LIMIT 1".format(experiment_id))
if cursor.rowcount == 0:
    print("No runs found")
else:
    entity=list(cursor)[0]
    run_uuid = entity[0]
    print("Last run id of '{}' experiment is: {}\n".format(experiment_name, run_uuid))
    print(entity)

#### Query metrics

In [ ]:
cursor.execute("SELECT * FROM metrics where run_uuid = '{}'".format(run_uuid))
if cursor.rowcount == 0:
    print("No metrics found")
else:
    for entry in cursor:
        print(entry)

### 2.5. List the artifacts in Cloud Storage

In [ ]:
!gsutil ls {MLFLOW_EXPERIMENTS_URI}/{experiment_id}/{run_uuid}/artifacts/model